In [11]:
result = """[[{'summary_text': 'Dr. Martin Luther King, Jr. spoke at the 1963 March on Washington. King: " 1963 is not an end but a beginning. There will be neither rest nor tranquility in America until the Negro has granted his citizenship rights" "We cannot walk alone. As we walk, we must make the plans we shall always turn back," he said. "We can never be satisfied as long as the Negro is the big faith in all the horrors of police brutality"" 1963 will go down in history as the greatest demonstration for freedom in the history of our nation," King said.'}], [{'summary_text': 'I say to you today, my friends, so even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream deeply rooted in the American dream. One day this nation will rise up and live out the true meaning of its creed. We hold these truths to be self-evident that all men are created equal. With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood.'}]] """
result = result.replace("[[{'summary_text': '", '').replace("'}], [{'summary_text': '", ' ').replace("'}]]", '')

print(result)

Dr. Martin Luther King, Jr. spoke at the 1963 March on Washington. King: " 1963 is not an end but a beginning. There will be neither rest nor tranquility in America until the Negro has granted his citizenship rights" "We cannot walk alone. As we walk, we must make the plans we shall always turn back," he said. "We can never be satisfied as long as the Negro is the big faith in all the horrors of police brutality"" 1963 will go down in history as the greatest demonstration for freedom in the history of our nation," King said. I say to you today, my friends, so even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream deeply rooted in the American dream. One day this nation will rise up and live out the true meaning of its creed. We hold these truths to be self-evident that all men are created equal. With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood. 


In [6]:
import transformers

undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-xsum-nl"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]

article = """
Volgens De Vries gaat het om "de hoogste beloning die ooit is uitgeloofd in Nederland". De stichting heeft een website waar donateurs geld kunnen storten, schrijft NH Nieuws. Volgens De Vries is dit initiatief ook bedoeld voor andere zaken waar beloningen voor een gouden tip worden uitgereikt. "Het is dus niet eenmalig", aldus De Vries. Het is de eerste keer dat zoiets wordt opgezet, stelt hij: De 18-jarige Tanja Groen verdween spoorloos tijdens de ontgroeningsweek van de Universiteit Maastricht in augustus 1993. Ze werd voor het laatst gezien nadat ze was vertrokken van een feestje. De studente zou vandaag 46 jaar zijn geworden. Ook de ouders van Groen waren op de persconferentie aanwezig. "Het is vandaag de verjaardag van Tanja Groen, die haar ouders al 27 jaar niet meer hebben kunnen vieren, omdat zij eind augustus 1993 spoorloos is verdwenen", zei De Vries. "Haar ouders zitten in tergende onzekerheid. Ze geloven dat ze niet meer leeft. Maar die ene promille vreet aan ze. Ze hebben recht op duidelijkheid. Ze komen op leeftijd. Grootste angst is nooit te weten wat er met hun kind is gebeurd." De Vries wil dat het miljoen binnen een jaar is ingezameld. Als het bedrag na een jaar lager uitkomt, dan is dat de uit te loven beloning. Is het meer, dan zal de rest van het geld gebruikt worden in beloningen in andere zaken. Het initiatief wordt gesteund door de politie en justitie. De afgelopen jaren is er vaker uitgebreid naar sporen van Tanja Groen gezocht, maar die zoekacties hebben niets concreets opgeleverd. Vorige week werd opnieuw naar de vrouw gezocht, op de Strabrechtse Heide in Noord-Brabant. Ook die zoektocht leverde niets op.
"""  # Dutch
summarization_pipeline(
    article,
    do_sample=True,
    top_p=0.75,
    top_k=50,
    min_length=50,
    early_stopping=True,
    truncation=True,
)[0]["summary_text"]

Your max_length is set to 1024, but you input_length is only 399. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=199)
/Users/rikjansen/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


'De vader van Tanja Groen, die 27 jaar geleden verdween, heeft een beloning van 1 miljoen pond uitgereikt om haar te helpen met haar verdwijning, zegt dat het de hoogste beloning die ooit is uitgeloofd in Nederland.'

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")

model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/Users/rikjansen/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Das Haus ist wunderbar.


/Users/rikjansen/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [1]:
%load_ext gradio

In [3]:
%%blocks

import gradio as gr

import whisper
from transformers import pipeline
from nltk import tokenize
from tokenizers import Tokenizer

def summarize(x):
    tokenizer = Tokenizer.from_pretrained("facebook/bart-large-cnn")

    model = whisper.load_model("small")
    result = model.transcribe(x, task="translate")

    #split text into sentences
    text = tokenize.sent_tokenize(result["text"])

    coll = []
    chunk = ""
    wordCount = 0
    #the maximum number of tokens the summarization model will accept at once
    n = 1024

    #create chunks of sentences to be summarized, each chunk not exceeding n tokens
    #needs an exception in case a single sentence exceeds n (however unlikely)
    for sent in text:
        wordCount += len(tokenizer.encode(sent))
        if wordCount <= n:
            chunk += sent
        else:
            coll.append(chunk)
            chunk = sent
            wordCount = 0
    coll.append(chunk)

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = []

    #summarize each chunk individually and add them together
    for val in coll:
        numTokens = len(tokenizer.encode(val))
        maxLength = min(500, numTokens)
        minLength = min(100, numTokens)
        summary.append(summarizer(val, max_length=maxLength, min_length=minLength, do_sample=False))


inp = gr.Audio(type='filepath')
button = gr.Button('Summarize')
out = gr.TextArea(label='Summary')

button.click(summarize, inputs = inp, outputs= out)

Running on local URL:  http://127.0.0.1:7860


Traceback (most recent call last):
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/gradio/routes.py", line 384, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1032, in process_api
    result = await self.call_function(
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 844, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/rikjansen/miniconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<string>", line 

In [19]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/users/rikjansen/downloads/gettysburg10.wav")

print(result["text"])

/Users/rikjansen/miniconda3/lib/python3.10/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Four score and seven years ago, our fathers brought forth on this continent a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal.


In [17]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
test = (summarizer(ARTICLE, max_length=500, min_length=500, do_sample=False))
print(test)


Your max_length is set to 500, but you input_length is only 489. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=244)


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men, and at one time, she was married to eight men at once. Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force. If convicted, she faces up to four years in prison. The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'sInvestigation Division. It was unclear whether any of the men will be prosecuted. The next court appearance is scheduled for May 18, according to her attorney, Christopher Wright, who declined to comment further on the case. It is not clear if any of her husbands have been charged with any crime in connection with the marriages,

In [19]:
import whisper
from transformers import pipeline
from nltk import tokenize
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("facebook/bart-large-cnn")

model = whisper.load_model("small")
result = model.transcribe("/users/rikjansen/downloads/NRCVandaag.mp3", task="translate")
print("Transcribed text: " + result["text"])

#split text into sentences
text = tokenize.sent_tokenize(result["text"])

coll = []
chunk = ""
wordCount = 0
#the maximum number of tokens the summarization model will accept at once
n = 1024

#create chunks of sentences to be summarized, each chunk not exceeding n tokens
#needs an exception in case a single sentence exceeds n (however unlikely)
for sent in text:
    wordCount += len(tokenizer.encode(sent))
    if wordCount <= n:
        chunk += sent
    else:
        coll.append(chunk)
        chunk = sent
        wordCount = 0
coll.append(chunk)

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = []

#summarize each chunk individually and add them together
for val in coll:
    numTokens = len(tokenizer.encode(val))
    maxLength = min(500, numTokens)
    minLength = min(100, numTokens)
    summary.append(summarizer(val, max_length=maxLength, min_length=minLength, do_sample=False))
            
print("Summarized text: ")
print(summary)


/Users/rikjansen/miniconda3/lib/python3.10/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text:  NPO Radio 1 podcast. The day. But how do we always get there after death? Why do we always come after death? We should be before death. That is the pastor who won ten bodies in the South-Ethelian church of Crotone. The turned migrants were told that their wooden boat was buried on the Middle East Sea. For 8000 euros they would make the crossing. But the rest are wooden planks that drive on waves and bodies that are buried in boxes in the local gym. Ten people are still missing. We don't find one child. He's five years old and we're looking for him. The migrant drama of this weekend has been the biggest shipwreck since times in Italy. Three smokers have been picked up. It has already given a walking discussion about boat refugees on New and Impulse. The extreme legal premier Meloni won elections with her promise to stop the rising number of migrants. But does that work? I ask the correspondent in Italy, Helene De Haans. Nice that you listen. My name is Marco Geiterbee

In [7]:
from nltk import tokenize
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("facebook/bart-large-cnn")
from transformers import pipeline

text = "I have the pleasure to present to you Dr. Martin Luther King. Yeah! I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation. Five score years ago, a great American in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity. But one hundred years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination one hundred years later. The Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity one hundred years later. The Negro is still languished in the corners of American society and finds himself an exile in his own land. So we've come here today to dramatize a shameful condition. In a sense we've come to our nation's capital to cash a check. In the architects of our republic, wrote the magnificent words of the Constitution and the Declaration of Independence. They were signing a promise or a note to whichever American was to fall out. This note was a promise at all men. Yes, black men as well as white men. To be guaranteed the unalienable rights of life, liberty and the pursuit of happiness. It is obvious today that America has defaulted on this promise or a note in so far as her citizens of color or concern. Instead of honoring this sacred obligation, America has given the Negro people a bad check. A check which has come back, Mark Insufficient Fund. But we refuse to believe that the Bank of Justice is bankrupt. We refuse to believe that I insufficient funds and the great bolts of opportunity of this nation. So we've come to cash this check. A check that will give us a pun demand, the riches of freedom and the security of justice. We have also come to this hell of spot to remind America of the fierce urgency of now. This is no time to engage in the luxury of cooling off or to take the tranquilizing drug of gradualism. Now is the time to make real the promises of democracy. Now is the time to rise from the dark and desolate valley of segregation to the sunlit path of racial justice. Now is the time. To lift our nation from the quick sense of racial injustice to the solid rock of brotherhood. Now is the time. It may have just a reality for all of God's children. It would be fatal for the nation to overlook the urgency of the moment. This sweltering summer of the Negroes legitimate discontent will not pass. Until that is an invigorating, autumn of freedom and equality. 1963 is not an end, but a beginning. Those who hope that the Negro needed to blow off steam and will now be content will have a root awakening if the nation returns to business as usual. Now will be neither rest nor tranquillity in America until the Negroes granted his citizenship rights. The whirlwinds of revolt will continue to shake the foundations of our nation until the bright day of justice emerges, but that is something that I must say to my people who stand on the wrong threshold which leads into the palace of justice. In the process of gaining our rightful place, we must not be guilty of wrongful deeds. Let us not seek the satisfy thirst for freedom by drinking from the cup of fiddles and hints. We must fail when the last struggle on the high trading of indigenous indiscence. We must not allow our creative protest to degenerate into physical violence. We must rise to the majestic heights of meeting physical force for soul force. The marvelous new militancy which has endowed the Negro community must not lead us to lay this trust of all white people. We may allow white folks as evidenced by their presence here today. If I'm realized that freedom is in a strictly planned plumber, we cannot walk alone. As we walk, we must make the pledge that we shall always march ahead. We cannot turn back. Now those who are asking the devotees of civil rights, when will you be satisfied? We can never be satisfied as long as the Negro is a great fit for all the young, spiritual horrors of police brutality. As long as our body is hidden within the legal travel, we cannot be enlarging in the motel to the highways and the hotels within the city. We cannot be satisfied as long as the Negro in Mississippi cannot vote. And the Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied and we will not be satisfied until just as rolls down like waters and righteousness like a mighty street. I am not unmindful that some of you have come here out of great trials and tribulations. Some of you have come fresh from now a jail sale. Some of you have come from areas where you are pressed quest for freedom. Left you battered by the storms of persecution and staggered by the winds of police brutality. You have been the veterans of creative suffering. Continue to work with the faith that online suffering is redemptive. Go back to Mississippi. Go back to Alabama. Go back to South Carolina. Go back to Georgia. Go back to Louisiana. Go back to the slums and get holes in our northern cities. Knowing that somehow this situation can and will be changed. Let us not wallow in the valley of despair. I say to you today, my friends. So even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream deeply rooted in the American dream. I have a dream that one day this nation will rise up and live out the true meaning of its creed. We hold these truths to be self evident that all men are created equal. I have a dream that one day on the Red Hills of Georgia, the sons of former slaves and the sons of former slave owners, will be able to sit down together at the table of brotherhood. I have a dream that one day even the state of Mississippi are state sweltering with the heat of injustice, sweltering with the heat of oppression and be transformed into an oasis of freedom and justice. I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character. I have a dream to be. I have a dream that one day down in Alabama with its vicious racists, with its governor having his lips tripping with the words of interposition and nullification. One day right there in Alabama, little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers. I have a dream today. I have a dream that one day every valor shall be exalted. Every hill in mounting shall be made low, the rough places will be made plain and the crooked places will be made straight and the go of the Lord shall be revealed and all flesh shall see it together. This is the faith that I go back to the south with, with this faith. We will be able to hue out of the mountain of despair, a stone of hope. With this faith, we will be able to transform the dangling discords of our nation into a beautiful symphony of brotherhood. With this faith, we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day. This will be the day, this will be the day when all of God's children will be able to sing with new meaning, my country tears of thee. Sweet land of liberty of thee I sing. Land where my father's died, land of the pilgrim's pride. From every mountain side, let freedom ring in the Americas to be a great nation. This must become true. So let freedom ring from the prodigious hill tops of New Hampshire, let freedom ring from the mighty mountains of New York, let freedom ring from the heightening alligators of Pennsylvania, let freedom ring from the snow cap Rock is of Colorado, let freedom ring from the crevace of swaps of California. But not only that, let freedom ring from stone mountain of Georgia, let freedom ring from look out mountain of Tennessee, let freedom ring from every hill and whole hill of Mississippi, from every mountain side, let freedom ring and when this happens, when we allow freedom ring, when we let it ring from every village and every hamlet, from every state and every city, we will be able to speed up that day when all of God's children, black men and white men, Jews and Gentiles, Protestants and Catholics, will be able to join hands and sing in the words of the old Negro spiritual. So, free at last, free at last, take God Almighty, we are free at last."


#split text into sentences
text = tokenize.sent_tokenize(text)

coll = []
chunk = ""
wordCount = 0
n = 1024

#create chunks of sentences to be summarized, each chunk not exceeding n tokens
#needs an exception in case a single sentence exceeds n (however unlikely)
for sent in text:
    wordCount += len(tokenizer.encode(sent))
    if wordCount <= n:
        chunk += sent
    else:
        coll.append(chunk)
        chunk = sent
        wordCount = 0
coll.append(chunk)

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = []

for val in coll:
    summary.append(summarizer(val, max_length=500, min_length=100, do_sample=False))
            
print(summary)

[[{'summary_text': 'Dr. Martin Luther King: "America has defaulted on this promise or a note in so far as her citizens of color or concern" "We refuse to believe that the Bank of Justice is bankrupt" "This sweltering summer of the Negroes legitimate discontent will not pass until that is an invigorating, autumn of freedom and equality" "Now is the time to rise from the dark and desolate valley of segregation to the sunlit path of racial justice" "If I\'m realized that freedom is in a strictly planned plumber, we cannot walk alone"'}], [{'summary_text': "I have a dream that one day this nation will rise up and live out the true meaning of its creed. We hold these truths to be self evident that all men are created equal. With this faith, we will be able to transform the dangling discords of our nation into a beautiful symphony of brotherhood. This will be the day, when all of God's children will be can to sing with new meaning, my country tears of thee.Sweet land of liberty of thee I sin

In [6]:
from nltk import sent_tokenize
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("facebook/bart-large-cnn")

text = "I have the pleasure to present to you Dr. Martin Luther King. Yeah! I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation. Five score years ago, a great American in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity. But one hundred years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination one hundred years later. The Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity one hundred years later. The Negro is still languished in the corners of American society and finds himself an exile in his own land. So we've come here today to dramatize a shameful condition. In a sense we've come to our nation's capital to cash a check. In the architects of our republic, wrote the magnificent words of the Constitution and the Declaration of Independence. They were signing a promise or a note to whichever American was to fall out. This note was a promise at all men. Yes, black men as well as white men. To be guaranteed the unalienable rights of life, liberty and the pursuit of happiness. It is obvious today that America has defaulted on this promise or a note in so far as her citizens of color or concern. Instead of honoring this sacred obligation, America has given the Negro people a bad check. A check which has come back, Mark Insufficient Fund. But we refuse to believe that the Bank of Justice is bankrupt. We refuse to believe that I insufficient funds and the great bolts of opportunity of this nation. So we've come to cash this check. A check that will give us a pun demand, the riches of freedom and the security of justice. We have also come to this hell of spot to remind America of the fierce urgency of now. This is no time to engage in the luxury of cooling off or to take the tranquilizing drug of gradualism. Now is the time to make real the promises of democracy. Now is the time to rise from the dark and desolate valley of segregation to the sunlit path of racial justice. Now is the time. To lift our nation from the quick sense of racial injustice to the solid rock of brotherhood. Now is the time. It may have just a reality for all of God's children. It would be fatal for the nation to overlook the urgency of the moment. This sweltering summer of the Negroes legitimate discontent will not pass. Until that is an invigorating, autumn of freedom and equality. 1963 is not an end, but a beginning. Those who hope that the Negro needed to blow off steam and will now be content will have a root awakening if the nation returns to business as usual. Now will be neither rest nor tranquillity in America until the Negroes granted his citizenship rights. The whirlwinds of revolt will continue to shake the foundations of our nation until the bright day of justice emerges, but that is something that I must say to my people who stand on the wrong threshold which leads into the palace of justice. In the process of gaining our rightful place, we must not be guilty of wrongful deeds. Let us not seek the satisfy thirst for freedom by drinking from the cup of fiddles and hints. We must fail when the last struggle on the high trading of indigenous indiscence. We must not allow our creative protest to degenerate into physical violence. We must rise to the majestic heights of meeting physical force for soul force. The marvelous new militancy which has endowed the Negro community must not lead us to lay this trust of all white people. We may allow white folks as evidenced by their presence here today. If I'm realized that freedom is in a strictly planned plumber, we cannot walk alone. As we walk, we must make the pledge that we shall always march ahead. We cannot turn back. Now those who are asking the devotees of civil rights, when will you be satisfied? We can never be satisfied as long as the Negro is a great fit for all the young, spiritual horrors of police brutality. As long as our body is hidden within the legal travel, we cannot be enlarging in the motel to the highways and the hotels within the city. We cannot be satisfied as long as the Negro in Mississippi cannot vote. And the Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied and we will not be satisfied until just as rolls down like waters and righteousness like a mighty street. I am not unmindful that some of you have come here out of great trials and tribulations. Some of you have come fresh from now a jail sale. Some of you have come from areas where you are pressed quest for freedom. Left you battered by the storms of persecution and staggered by the winds of police brutality. You have been the veterans of creative suffering. Continue to work with the faith that online suffering is redemptive. Go back to Mississippi. Go back to Alabama. Go back to South Carolina. Go back to Georgia. Go back to Louisiana. Go back to the slums and get holes in our northern cities. Knowing that somehow this situation can and will be changed. Let us not wallow in the valley of despair. I say to you today, my friends. So even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream deeply rooted in the American dream. I have a dream that one day this nation will rise up and live out the true meaning of its creed. We hold these truths to be self evident that all men are created equal. I have a dream that one day on the Red Hills of Georgia, the sons of former slaves and the sons of former slave owners, will be able to sit down together at the table of brotherhood. I have a dream that one day even the state of Mississippi are state sweltering with the heat of injustice, sweltering with the heat of oppression and be transformed into an oasis of freedom and justice. I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character. I have a dream to be. I have a dream that one day down in Alabama with its vicious racists, with its governor having his lips tripping with the words of interposition and nullification. One day right there in Alabama, little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers. I have a dream today. I have a dream that one day every valor shall be exalted. Every hill in mounting shall be made low, the rough places will be made plain and the crooked places will be made straight and the go of the Lord shall be revealed and all flesh shall see it together. This is the faith that I go back to the south with, with this faith. We will be able to hue out of the mountain of despair, a stone of hope. With this faith, we will be able to transform the dangling discords of our nation into a beautiful symphony of brotherhood. With this faith, we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day. This will be the day, this will be the day when all of God's children will be able to sing with new meaning, my country tears of thee. Sweet land of liberty of thee I sing. Land where my father's died, land of the pilgrim's pride. From every mountain side, let freedom ring in the Americas to be a great nation. This must become true. So let freedom ring from the prodigious hill tops of New Hampshire, let freedom ring from the mighty mountains of New York, let freedom ring from the heightening alligators of Pennsylvania, let freedom ring from the snow cap Rock is of Colorado, let freedom ring from the crevace of swaps of California. But not only that, let freedom ring from stone mountain of Georgia, let freedom ring from look out mountain of Tennessee, let freedom ring from every hill and whole hill of Mississippi, from every mountain side, let freedom ring and when this happens, when we allow freedom ring, when we let it ring from every village and every hamlet, from every state and every city, we will be able to speed up that day when all of God's children, black men and white men, Jews and Gentiles, Protestants and Catholics, will be able to join hands and sing in the words of the old Negro spiritual. So, free at last, free at last, take God Almighty, we are free at last."

#split text into sentences
text = sent_tokenize(text)

coll = []
chunk = ""
wordCount = 0
n = 1024

#create groups of sentences to be summarized
#needs an exception in case a single sentence exceeds n (however unlikely)
for sent in text:
    wordCount += len(tokenizer.encode(sent))
    if wordCount <= n:
        chunk += sent
    else:
        coll.append(chunk)
        chunk = sent
        wordCount = 0
coll.append(chunk)

print(len(coll))
print(coll[0])
print(coll[1])

2
I have the pleasure to present to you Dr. Martin Luther King.Yeah!I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation.Five score years ago, a great American in whose symbolic shadow we stand today, signed the Emancipation Proclamation.This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice.It came as a joyous daybreak to end the long night of their captivity.But one hundred years later, the Negro still is not free.One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination one hundred years later.The Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity one hundred years later.The Negro is still languished in the corners of American society and finds himself an exile in his own land.So we've come here

In [6]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("facebook/bart-large-cnn")

text =  """ FBI Director Christopher Wray on Tuesday acknowledged that the bureau believes the Covid-19 pandemic was likely the result of a lab accident in Wuhan, China.

In his first public comments on the FBI’s investigation into the virus’ origins during an interview with Fox News, Wray said that “the FBI has for quite some time now assessed that the origins of the pandemic are most likely a potential lab incident in Wuhan.”
"""

print(len(tokenizer.encode(text)))

100
